In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import seaborn as sns
from itertools import product
from itertools import combinations_with_replacement
import torch
import time
from torch.autograd import Variable
%matplotlib inline

In [9]:
from utils import *
# %load_ext autoreload
# %autoreload 2

In [85]:
adult = pd.read_csv('../labels.txt', delimiter='\t', header=0, names=['user_id','website','rating'])
# trec = pd.read_csv('../trec-rf10-crowd/trec-rf10-data.txt', delimiter='\t')

# r = pd.Series([2,3,2,3], index=[1,2,0,-2])
# trec['label_bin'] = trec['label'].map(r)
# trec.head(10)

In [86]:
testframe = create_user_task_ids(adult, 'user_id', 'website', 'rating')
# testframe = create_user_task_ids(trec, 'workerID', 'docID', 'label_bin')
testframe.head()

,user_id,website,rating,task_id,uid,bin
0,A1PXXEOGQ76RNJ,http://000.cc,G,0,0,2
1,A1PXXEOGQ76RNJ,http://000.cc,G,0,0,2
2,A21US576U8SCO4,http://000.cc,G,0,1,2
3,A2LGX47NN7C5D3,http://000.cc,G,0,2,2
4,A2OVKTB7VNY8EW,http://000.cc,G,0,3,2


In [92]:
completed, values, ind = compute_individual_dist(testframe)

In [116]:
start = time.time()
features = np.empty((ind.shape[0], ind.shape[1]**2*ind.shape[0]))
delta_matrices_all = np.empty((ind.shape[0], ind.shape[0], ind.shape[1], ind.shape[1]))
score_matrices_all = np.empty((ind.shape[0], ind.shape[0], ind.shape[1], ind.shape[1]))
for user_index in range(values.shape[0]):
    print(user_index)
    #check for full joint distribution or add a prior later
    if np.sum(ind[user_index]==0) > 0:
        continue
    #compute delta matrices with all other users where applicable
    else:
        #create a mask so that other half of tasks can be used later to find score matrix
        mask = np.random.randint(0,2,values.shape).astype(bool)
        delta_matrices, t_m_i_1, cluster_img = compute_deltas(user_index, completed, values, ind, mask, False, 20)
        features[user_index,:] = cluster_img.flatten()
        delta_matrices_all[user_index,:,:,:] = delta_matrices
        score_matrices, t_m_i_2 = compute_deltas(user_index, completed, values, ind, ~mask, True, 20)
        score_matrices_all[user_index,:,:,:] = score_matrices
#         print(np.sum(t_m_i_1), np.sum(t_m_i_2))
#         if len(np.intersect1d(np.array(np.where(t_m_i_1==True)), np.array(np.where(t_m_i_2==True))))>0:
#             print(np.intersect1d(np.array(np.where(t_m_i_1==True)), np.array(np.where(t_m_i_2==True))))
#             print(regret(score_matrices, delta_matrices, \
#                      np.logical_and((t_m_i_1==True), (t_m_i_2==True))))
print(time.time()-start)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [129]:
np.sum(np.isnan(features), axis=0)
features = features[:,np.sum(np.isnan(features), axis=0)<features.shape[0]]
print(features.shape)

(389, 1664)


In [130]:
from sklearn.cluster import KMeans

def kmeans_missing(X, n_clusters, max_iter=10):
    """Perform K-Means clustering on data with missing values.

    Args:
      X: An [n_samples, n_features] array of data to cluster.
      n_clusters: Number of clusters to form.
      max_iter: Maximum number of EM iterations to perform.

    Returns:
      labels: An [n_samples] vector of integer labels.
      centroids: An [n_clusters, n_features] array of cluster centroids.
      X_hat: Copy of X with the missing values filled in.
    """

    # Initialize missing values to their column means
    missing = ~np.isfinite(X)
    mu = np.nanmean(X, 0, keepdims=1)
    X_hat = np.where(missing, mu, X)

    for i in range(max_iter):
        if i > 0:
            # initialize KMeans with the previous set of centroids. this is much
            # faster and makes it easier to check convergence (since labels
            # won't be permuted on every iteration), but might be more prone to
            # getting stuck in local minima.
            cls = KMeans(n_clusters, init=prev_centroids)
        else:
            # do multiple random initializations in parallel
            cls = KMeans(n_clusters, n_jobs=-1)

        # perform clustering on the filled-in data
        labels = cls.fit_predict(X_hat)
        centroids = cls.cluster_centers_

        # fill in the missing values based on their cluster centroids
        X_hat[missing] = centroids[labels][missing]

        # when the labels have stopped changing then we have converged
        if i > 0 and np.all(labels == prev_labels):
            break

        prev_labels = labels
        prev_centroids = cls.cluster_centers_

    return labels, centroids, X_hat

In [188]:
labels, centroids, X_hat = kmeans_missing(features, 40, max_iter=20)
print(labels)

[36 29  0  0 15 12  0  2  4 18  0  6  6 29 16  0 18  5  0  0 37 20  7 22  6
  7  0 13  0 32  1  0  0  0 14  0  0 24  0  0  0 18 29  0  3 21 19 35  0 27
  0  0  0  0  0  0  0  0 10 11  6  7  0  0  0  0 39 29  0 17  7  0  0  9  0
  0 18  0  8  0  0  0  0  0  0  0  6  0  0  6  0  7  0  7  0 31  0  0 23  0
 25  0  0  0  0  7  0  6  0  0  0  0 27  6 29  0  0  0 11 29 33  0 11 25  6
  0  6  0  7 28 29 29  0  7  0 26  0  6  0  0  0  0  0  0  0  0  0 29  0  6
  0  0  0  0  0  0  0  0 29  0  6  0  0  0  6  0  0  0  0 29  0  0  6  0  0
  0  0  0  7  0  0  0 34  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  6  7
  0  0  0  6  0  0  0  0  0 38  0  0 29  0  0  0  0  0  0  0  0  0 38  0  0
  0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 30  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 23  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0

/Users/annahilgard/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:893: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


In [192]:
def calc_cluster_scoring_matrices(labels, delta_matrices_all):
    delta_matrices_new=np.zeros_like(delta_matrices_all)
    for i in np.unique(labels):
#         print(np.sum(labels==i))
        for j in np.unique(labels):
#             print(np.sum(labels==j))
#             print(i)
            missing = ~np.isfinite(delta_matrices_all)
            mu = np.nanmean(delta_matrices_all, 0, keepdims=0)
            filled = np.where(missing, mu, delta_matrices_all)
            cluster_i = np.average(delta_matrices_all[labels==i], axis=0)
            if np.sum(np.sum(np.isnan(cluster_i), (1,2)))>0:
#                 print(i,j)
                missing = ~np.isfinite(delta_matrices_all)
                mu = np.nanmean(delta_matrices_all, 0, keepdims=1)
                X_hat = np.where(missing, mu, X)
#             print(cluster_i.shape)
            cluster_j = np.average(cluster_i[labels==j], axis=0)
#             print(cluster_j.shape)
#             print(delta_matrices_new[np.ix_(labels==i,labels==j)].shape)
            delta_matrices_new[np.ix_(labels==i,labels==j)] = cluster_j
    score_matrices_new = np.sign(delta_matrices_new)
    print(np.sum(score_matrices_new==1))
    score_matrices_new[score_matrices_new==-1]=0
    return score_matrices_new

In [193]:
calc_cluster_scoring_matrices(labels, delta_matrices_all)

TypeError: an integer is required (got type tuple)